<a href="https://colab.research.google.com/github/EN10/modded-llama2.c/blob/main/Baby_Llama_128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
# training and inference code
!git clone https://github.com/eniompw/modded-llama2.c

Cloning into 'modded-llama2.c'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 32 (delta 10), reused 11 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 56.99 KiB | 5.18 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [69]:
!mkdir -p ./modded-llama2.c/data
# Download files in parallel directly into the target directory
!wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.model & \
  wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/raw/main/tok128/tok128.vocab & \
  wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.tar.gz & \
  wait

# Untar the large file after all downloads are complete
!tar -I pigz -xf ./modded-llama2.c/data/tok128.tar.gz -C ./modded-llama2.c/data/

# compile run / inference executable
!cd modded-llama2.c && gcc -O3 -o run run.c -lm
# create tok105.bin
!cd modded-llama2.c && python tokenizer.py --tokenizer-model=data/tok128.model

--2025-05-29 09:40:44--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.tar.gz
--2025-05-29 09:40:44--  https://huggingface.co/datasets/enio/TinyStories/raw/main/tok128/tok128.vocab
Resolving huggingface.co (huggingface.co)... Resolving huggingface.co (huggingface.co)... --2025-05-29 09:40:44--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.model
Resolving huggingface.co (huggingface.co)... 3.163.189.114, 3.163.189.37, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.114|:443... 3.163.189.90, 3.163.189.37, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
connected.
3.163.189.90, 3.163.189.74, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... HTTP request sent, awaiting response... connected.
HTTP request sent, awaiting response... HTTP request sent, awaiting response... 200 OK
Length: 874 [text/plain]
Saving to: ‘./modded

In [70]:
# train neural net
!cd modded-llama2.c && python train.py --vocab_source=custom --vocab_size=128 --compile=False \
  --dim=128 --n_layers=5 --n_heads=8 --n_kv_heads=4 --batch_size=32 \
  --always_save_checkpoint=True --eval_interval=100 --max_iters=100 #--init_from='resume'

Overriding: vocab_source = custom
Overriding: vocab_size = 128
Overriding: compile = False
Overriding: dim = 128
Overriding: n_layers = 5
Overriding: n_heads = 8
Overriding: n_kv_heads = 4
Overriding: batch_size = 32
Overriding: always_save_checkpoint = True
Overriding: eval_interval = 100
Overriding: max_iters = 100
Starting run in out
Vocab size (original/padded): 128/128
Tokens per iteration: 65,536
Max iterations: 100
Compiling: False
Initializing a new model from scratch
Optimizer groups:
  Group 0: 16,384 params, LR_mult=1.00, WD=0.0
  Group 1: 1,408 params, LR_mult=0.10, WD=0.0
  Group 2: 921,600 params, LR_mult=1.00, WD=0.1
Total optimizable parameters: 939,392
Using fused AdamW: True
Created a PretokDataset with rng seed 42
Starting training loop...
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
step 0: train loss 4.9223, val loss 4.9226
New best val loss: 4.9226
Saving checkpoint to out/ckpt.pt
wrote out/model.bin
0 | loss 4.9256 | lr 5.00e-

In [72]:
# inference
!cd ./modded-llama2.c && ./run out/model.bin -z data/tok128.bin -t 0.8 -n 256 -i "Once upon a time "

Once upon a time pap and stappy bid a log."<unk>Tied to o wels a dererd thacked it. The made eadny. Saponed. Tis nos nory crols and days. Mis it ot cat ne save. Thoom mamed to gor the barle tige, "<unk>" Lit and ness as hady. Sue clen's none the tie, uts and thark to she tise, nory, Sile and saik. Mold saw a loat gallld a loplr d
achieved tok/s: 913.978495
